<a href="https://colab.research.google.com/github/uddeshya-23/Clustering-Data-analysis/blob/main/K_Means_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ONLINE RETAIL DATA SET ANALYSIS

[Online retail dataset](https://www.kaggle.com/datasets/mashlyn/online-retail-ii-uci)



*   Read The Data
*   Visualize Data
*   Prepare Data for Modelling
*   Modelling
*   Final Analysis and Recommendation




In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [46]:
##READ THE DATASET
retail=pd.read_csv('/content/drive/MyDrive/Upgrad Data/Online+Retail.csv', sep=',',encoding='ISO-8859-1', header=0)
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom


In [47]:
##SHAPE OF DATASET
retail.shape

(541909, 8)

In [48]:
retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


# **CLEANING THE DATA**







In [49]:
## Missing Value
round(100*(retail.isnull().sum()) / len(retail), 2)

InvoiceNo       0.00
StockCode       0.00
Description     0.27
Quantity        0.00
InvoiceDate     0.00
UnitPrice       0.00
CustomerID     24.93
Country         0.00
dtype: float64

In [50]:
## Drop Missing Columns
retail=retail.dropna()

In [51]:
retail.shape

(406829, 8)

# **Prepare data for modelling**

- (R) Recency : Number of days since last purchase

- (F) Frequency : Number of transaction

- (M) Monetary : Total amount of transaction (revenue contributed)

In [52]:
## Start with Monetary the easy one  so for that we need column amount contributed by each customer

retail['amount']=retail['Quantity']*retail['UnitPrice']
retail.head()

<ipython-input-52-c26a6a9ce073>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retail['amount']=retail['Quantity']*retail['UnitPrice']


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom,20.34


In [53]:
##Monetary
grouped_retail=retail.groupby('CustomerID')['amount'].sum()
grouped_retail=grouped_retail.reset_index()
grouped_retail.head()

,CustomerID,amount
0,12346.0,0.00
1,12347.0,4310.00
2,12348.0,1797.24
3,12349.0,1757.55
4,12350.0,334.40


In [54]:
##Frequency
frequency=retail.groupby('CustomerID')['InvoiceNo'].sum()
frequency=frequency.reset_index()
frequency.columns=['CustomerID', 'InvoiceNo']
frequency.head()

,CustomerID,InvoiceNo
0,12346.0,541431C541433
1,12347.0,5376265376265376265376265376265376265376265376...
2,12348.0,5393185393185393185393185393185393185393185393...
3,12349.0,5776095776095776095776095776095776095776095776...
4,12350.0,5430375430375430375430375430375430375430375430...


In [55]:
##MERGE BOTH "grouped_retail" &, "frequency" on "CUSTOMERID"
grouped_retail=pd.merge(grouped_retail, frequency, on='CustomerID', how='inner')
grouped_retail.head()

,CustomerID,amount,InvoiceNo
0,12346.0,0.00,541431C541433
1,12347.0,4310.00,5376265376265376265376265376265376265376265376...
2,12348.0,1797.24,5393185393185393185393185393185393185393185393...
3,12349.0,1757.55,5776095776095776095776095776095776095776095776...
4,12350.0,334.40,5430375430375430375430375430375430375430375430...


In [56]:
##RECENCY
##FOR this we need to convert invoicedate into datetime object of pandas to do the arithmetic operation on datetime to get the recency

retail['InvoiceDate']=pd.to_datetime(retail['InvoiceDate'], format='%d-%m-%Y %H:%M')
retail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406829 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    406829 non-null  object        
 1   StockCode    406829 non-null  object        
 2   Description  406829 non-null  object        
 3   Quantity     406829 non-null  int64         
 4   InvoiceDate  406829 non-null  datetime64[ns]
 5   UnitPrice    406829 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      406829 non-null  object        
 8   amount       406829 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 31.0+ MB


In [57]:
##Compute the max date
max_date=max(retail['InvoiceDate'])
max_date

Timestamp('2011-12-09 12:50:00')

In [58]:
## Compute the Difference
retail['period']=max_date-retail['InvoiceDate']
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount,period
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,373 days 04:24:00
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,373 days 04:24:00
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,373 days 04:24:00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,373 days 04:24:00
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,373 days 04:24:00


In [59]:
last_purchase=retail.groupby('CustomerID')['period'].min()
last_purchase=last_purchase.reset_index()
last_purchase.head()

,CustomerID,period
0,12346.0,325 days 02:33:00
1,12347.0,1 days 20:58:00
2,12348.0,74 days 23:37:00
3,12349.0,18 days 02:59:00
4,12350.0,309 days 20:49:00


In [60]:
##MERGE BOTH "grouped_retail" , "frequency" &, "last_purchase" on "CUSTOMERID"
grouped_retail=pd.merge(grouped_retail, last_purchase, on='CustomerID', how='inner')
grouped_retail.columns=['CustomerID', 'amount', 'InvoiceNo', 'period']
grouped_retail.head()

,CustomerID,amount,InvoiceNo,period
0,12346.0,0.00,541431C541433,325 days 02:33:00
1,12347.0,4310.00,5376265376265376265376265376265376265376265376...,1 days 20:58:00
2,12348.0,1797.24,5393185393185393185393185393185393185393185393...,74 days 23:37:00
3,12349.0,1757.55,5776095776095776095776095776095776095776095776...,18 days 02:59:00
4,12350.0,334.40,5430375430375430375430375430375430375430375430...,309 days 20:49:00


In [61]:
grouped_retail.columns

Index(['CustomerID', 'amount', 'InvoiceNo', 'period'], dtype='object')